# [Problem](https://thefiddler.substack.com/p/can-you-shut-the-box)

_In a simplified version of Shut the Box, there are six initially unflipped tiles, numbered 1-6, on the side of a box. You roll a fair, six-sided die, after which you can flip down any combination of unflipped tiles that add to the roll. For example, if you roll a 5, you could flip down the 5, the 1 and the 4, or the 2 and the 3. You proceed until no set of flips is possible (e.g., you rolled a 3 but your only remaining tiles are 2 and 6). To win, you must flip down all six tiles._

_Assuming you play with an optimal strategy (i.e., maximizing your chances of winning after any given roll), what is the probability that you’ll win?_

# Solution

First things first, if we want to find the probability, we'll first need to first determine optimal play. Intuitively, we should always try to flip the tile of the number we rolled (i.e. largest possible tiles we can flip) since the larger numbers are harder to flip (6 can only be flipped if you roll a 6 while 1 can be flipped on any roll except for a 2). But what if the number you rolled isn't available? While I'm not sure of this, I think the optimality of the choice is only dependent on the number of tiles you 

In order to win, we know we must